## Data Modeling
***

Notes
1. Dummy
2. Logreg

XGBClassifier, RandomForest

3. XGBClassifier, f2
3. XGBClassifier, f2-weighted
3. XGBClassifier, f2-SMOTE
3. XGBClassifier, f2-ADASYN

Randomize winner?

Gridsearch from there?

3. XGBClassifier, f2 - randomizedcv
3. XGBClassifier, f2-weighted - randomizedcv
3. XGBClassifier, f2-SMOTE - randomizedcv
3. XGBClassifier, f2, randomizedcv
3. XGBClassifier, f2-weighted, randomizedcv
3. XGBClassifier, f2, randomizedcv

In [38]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score, make_scorer, fbeta_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImPipeline
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, log_loss, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, train_test_split, RepeatedStratifiedKFold, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

df = pd.read_csv('data/Cardiovascular_Diseases_Risk_Prediction_Dataset.csv')
ordinal = ['General_Health', 'Checkup', 'Age_Category']
numeric = list(df.select_dtypes(exclude=object).columns)
categorical = list(df.select_dtypes(object).columns)
for i in ordinal:
    categorical.remove(i)
categorical.remove('Heart_Disease')

# Splitting the training from the validation data
# Making sure the split is stratefied given teh imbalance of our target variable
y = df['Heart_Disease']
X = df.drop('Heart_Disease',axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=12, stratify= y)

# Transforming the target variable into 1's and 0's
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)


# Listing categories in order for each ordinal variable.
health = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']
check = [
    'Never', '5 or more years ago', 'Within the past 5 years',
    'Within the past 2 years', 'Within the past year'
]
age = [
    '18-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
    '60-64', '65-69', '70-74', '75-79', '80+'
]

# Instantiating an OrdinalEncoder transformer to encode ordinal variables. 
oe = OrdinalEncoder(categories=[health, check, age])


# Instantiating a OneHotEncoder transformer to be used on the categorical varaibles. 
ohe = OneHotEncoder()

# Creating a column transformer to be used in a pipeline
col_transformer = ColumnTransformer(transformers=[
    ('oe', OrdinalEncoder(categories=[health, check, age]), ordinal),
    ('ohe', OneHotEncoder(), categorical)
], remainder="passthrough")

def model_scores(model_name, model, X_train, y_train, model_list= [], notes = ''):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12)
    scoring = {'f2': make_scorer(fbeta_score, beta=2),
               'recall': 'recall',
               'accuracy':'accuracy',
               'f1':'f1',
               'roc_auc':'roc_auc',
               'precision': 'precision'}
    scores = cross_validate(model,
                           X_train,
                           y_train,
                           scoring = scoring,
                           cv=skf,
                           n_jobs=-1)
    time = round(scores['fit_time'].mean() + scores['score_time'].mean(),0)
    f2 = round(scores['test_f2'].mean(),4)*100
    recall = round(scores['test_recall'].mean(),4)*100
    accuracy = round(scores['test_accuracy'].mean(),4)*100
    f1 = round(scores['test_f1'].mean(),4)*100
    roc_auc = round(scores['test_roc_auc'].mean(),4)*100
    precision = round(scores['test_precision'].mean(),4)*100
    model_list.append([model_name, time, f2, recall, accuracy, f1, roc_auc, precision, notes])
    df = pd.DataFrame(model_list, columns=['model', 'time (in s)', 'f2', 'recall', 'accuracy', 'f1', 'roc_auc', 'precision', 'notes'])
    return model_list, df


In [51]:
def model_scores(model, X, y, model_list= [], cv= 5, model_name = ''):
    if cv>1:
        skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=12)
        scoring = {'f2': make_scorer(fbeta_score, beta=2),
                   'accuracy':'accuracy',
                   'precision': 'precision',
                   'recall': 'recall',
                   'roc_auc':'roc_auc',
                   'log_loss': 'neg_log_loss'}
        scores = cross_validate(model,
                               X,
                               y,
                               scoring = scoring,
                               cv=skf,
                               n_jobs=-1)
        f2 = round(scores['test_f2'].mean(),4)*100
        accuracy = round(scores['test_accuracy'].mean(),4)*100
        precision = round(scores['test_precision'].mean(),4)*100
        recall = round(scores['test_recall'].mean(),4)*100
        roc_auc = round(scores['test_roc_auc'].mean(),4)*100
        lg_loss = round(scores['test_log_loss'].mean(),4)*100
    else:
        y_preds = model.predict(X)
        f2 = round(fbeta_score(y,y_preds,beta=2),4)*100
        recall = round(recall_score(y,y_preds),4)*100
        accuracy = round(accuracy_score(y,y_preds),4)*100
        roc_auc = round(roc_auc_score(y,y_preds),4)*100
        precision = round(precision_score(y,y_preds, zero_division=0.0),4)*100
        lg_loss = round(log_loss(y,model.predict_proba(X)),4)*100
    model_list.append([model_name, f2, accuracy, precision, recall, roc_auc, lg_loss])
    df = pd.DataFrame(model_list, columns=['name', 'f2', 'accuracy', 'precision', 'recall',  'roc_auc', 'log_loss'])
    return model_list, df

### Dummy Model

In [52]:
pipe = Pipeline(steps=[('ct', col_transformer),
                      ('dm', DummyClassifier(strategy='most_frequent'))])
dummy_model = pipe.fit(X_train, y_train)
ml, mdf = model_scores(dummy_model, X_train, y_train, model_list= [], cv= 5, model_name = 'Dummy Model')
mdf

,name,f2,accuracy,precision,recall,roc_auc,log_loss
0,Dummy Model,0.0,91.92,0.0,0.0,50.0,-291.41


### Baseline Model

In [53]:
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                      ('log_reg', LogisticRegression(solver='lbfgs', max_iter=1000, random_state=12))])
logreg_model = pipe.fit(X_train, y_train)
ml, mdf = model_scores(logreg_model, X_train, y_train, model_list= ml, cv= 5, model_name = 'Baseline Model')
mdf

,name,f2,accuracy,precision,recall,roc_auc,log_loss
0,Dummy Model,0.0,91.92,0.00,0.00,50.00,-291.41
1,Baseline Model,7.5,91.94,51.39,6.18,83.41,-22.23


### Model Iteration


In [58]:
pipe_rfc = Pipeline(steps=[('ct', col_transformer),
                       ('rfc', RandomForestClassifier(random_state=12))])
rfc_model = pipe_rfc.fit(X_train, y_train)
compare_list, compare_df = model_scores(rfc_model, X_train, y_train, model_list= [], model_name= 'Basic RandomForestClassifier')

pipe_xgb = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12, tree_method= 'hist'))])
xgb_model = pipe_xgb.fit(X_train, y_train)
compare_list, compare_df = model_scores(xgb_model, X_train, y_train, model_list= compare_list, model_name= 'Basic XGBClassifier')
compare_df


,name,f2,accuracy,precision,recall,roc_auc,log_loss
0,Basic RandomForestClassifier,5.34,91.81,43.76,4.38,80.85,-29.00
1,Basic XGBClassifier,6.54,91.87,47.73,5.38,82.94,-22.47


In [59]:
# Choose between XGBClassifier Variants

# Basic
ml_xgb, df_xgb = model_scores(xgb_model, X_train, y_train, model_list = [], model_name= "Basic XGB")

# Weighted
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xgb', XGBClassifier(random_state=12,scale_pos_weight=283883/24971))])
xgb_weighted = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores(xgb_weighted, X_train, y_train, model_list = ml_xgb, model_name= "Weighted XGB")

# SMOTE
pipe = ImPipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                       ('sm',SMOTE(random_state=12)),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_smote = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores(xgb_smote, X_train, y_train, model_list = ml_xgb, model_name= "SMOTE XGB")

# ADASYN
pipe = ImPipeline(steps=[('ct', col_transformer),
                       ('ss', StandardScaler()),
                       ('sm',ADASYN(random_state=12)),
                       ('xgb', XGBClassifier(random_state=12))])
xgb_adasyn = pipe.fit(X_train, y_train)
ml_xgb, df_xgb = model_scores(xgb_adasyn, X_train, y_train, model_list = ml_xgb, model_name= "ADASYN XGB")
df_xgb


,name,f2,accuracy,precision,recall,roc_auc,log_loss
0,Basic XGB,6.54,91.87,47.73,5.38,82.94,-22.47
1,Weighted XGB,49.27,74.52,20.62,75.49,82.41,-47.16
2,SMOTE XGB,6.75,91.81,44.54,5.57,82.81,-22.56
3,ADASYN XGB,5.84,91.78,42.56,4.81,82.82,-22.55


In [14]:
# Randomize the Weighted
pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xbg', XGBClassifier(random_state=12, tree_method='hist', scale_pos_weight= 283883/24971))])

params = {
    'xbg__n_estimators': range(50,1000,50),
    'xbg__max_depth': range(1,15),
    'xbg__eta': [0.001, 0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3],
    'xbg__colsample_bytree': np.linspace(0,1,50),
    'xbg__min_child_weight': range(1,10),
    'xbg__gamma': [0, 0.1, 1, 10, 100, 1000],
    'xbg__reg_alpha': [0, 0.01, 0.1, 1, 10],
    'xbg__reg_lambda':[0, 0.01, 0.1, 1, 10]
}
ftwo_scorer = make_scorer(fbeta_score, beta=2)

rs = RandomizedSearchCV(estimator= pipe, param_distributions= params,
                       n_iter=500, scoring=ftwo_scorer, n_jobs=-1, cv=5, random_state=12)
rand_search = rs.fit(X_train, y_train)
rand_model = rand_search.best_estimator_
ml, df1 = model_scores('XGBClassifier', rand_model, X_train, y_train, model_list= [], notes = 'Best Estimator from RS')
print(rand_search.best_params_)
df1


{'xbg__reg_lambda': 10, 'xbg__reg_alpha': 0.1, 'xbg__n_estimators': 650, 'xbg__min_child_weight': 1, 'xbg__max_depth': 4, 'xbg__gamma': 10, 'xbg__eta': 0.15, 'xbg__colsample_bytree': 0.42857142857142855}


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,7.0,50.32,80.33,72.72,32.25,83.63,20.18,Best Estimator from RS


In [16]:
# GridSearching the winnfer for the final
xgb = XGBClassifier(reg_lambda= 10, 
              reg_alpha= 0.1,
              min_child_weight= 1,
              gamma= 10,
              colsample_bytree= 0.42857142857142855,
              tree_method= "hist",
              scale_pos_weight=283883/24971,
              random_state=12)

pipe = Pipeline(steps=[('ct', col_transformer),
                       ('xbg', xgb)])

params = {'xbg__n_estimators': np.linspace(400,800,10,dtype=int),
          'xbg__max_depth': [2,3,4,5,6,7],
          'xbg__eta': np.linspace(0.1,0.2,10)
         }

gs = GridSearchCV(pipe, param_grid= params, scoring=ftwo_scorer, n_jobs=-1, cv=5, verbose= 3)

g_search = gs.fit(X_train, y_train)
gs_model = g_search.best_estimator_
ml, df1 = model_scores('XGBClassifier', gs_model, X_train, y_train, model_list= [], notes = 'Best Estimator from GS')
print(g_search.best_params_)
df1


Fitting 5 folds for each of 600 candidates, totalling 3000 fits
{'xbg__eta': 0.1, 'xbg__max_depth': 2, 'xbg__n_estimators': 400}


,model,time (in s),f2,recall,accuracy,f1,roc_auc,precision,notes
0,XGBClassifier,5.0,50.46,80.44,72.82,32.37,83.65,20.26,Best Estimator from GS


### Final Model


In [55]:
from sklearn.metrics import accuracy_score, precision_score, log_loss
models_dic = {'Dummy Model': dummy_model,
             'Baseline Model': logreg_model,
             'Final Model':gs_model}
ml=[]
for key in models_dic:
    model = models_dic[key]
    ml, mdf = model_scores(model, X_test, y_test, model_list= ml, cv= 0, model_name = key)
mdf

,name,f2,accuracy,precision,recall,roc_auc,log_loss
0,Dummy Model,0.00,91.91,0.00,0.00,50.00,291.42
1,Baseline Model,7.70,91.89,48.65,6.36,52.88,22.09
2,Final Model,50.59,72.71,20.26,80.87,76.43,50.70
